**<center><h1>Music Recommendation Based on Rhythmic Similarity Using Locality-Sensitive Hashing (LSH)</h1></center>**

### Objective:

To implement Locality-Sensitive Hashing (LSH) on audio data and evaluate its performance for duplicate detection.

#### Part #1:

##### Question #1:

In [1]:
#   Importing the necessary modules/libraries.

import os
import librosa
import librosa.display
from IPython.display import Audio
from tqdm import tqdm
import numpy as np
import pandas as pd
import annoy
from scipy.spatial.distance import cosine
from sklearn.cluster import KMeans

#   Ignoring any warnings.

import warnings
warnings.filterwarnings("ignore")

- **Performing pre-processing on the audio dataset (extracting the Mel-Frequency Cepstral Coefficients (MFCC) features):**

In [2]:
#   Function to extract and return relevant features from the audio files.

def extract_features(file_name):
    audio, sample_rate=librosa.load(file_name, res_type="kaiser_fast")  #   Loading the audio file.
    mfcc=librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)   #   Extracting the Mel-Frequency Cepstral Coefficients (MFCC) features.
    mfcc_scaled=np.mean(mfcc.T, axis=0) #   Scaling the Mel-Frequency Cepstral Coefficients (MFCC) features.
    return mfcc_scaled

In [3]:
folder=r"AudioFiles"
feature_dataframe=pd.DataFrame(columns=["Feature", "Label"])    #   Creating a pandas.DataFrame to store the features and corresponding labels.
for sub_folder in tqdm(os.listdir(folder)): #   Iterating through the sub-folders in the folder.
    if sub_folder==".DS_Store": #   Checking whether the sub-folder is a hidden file or not.
        continue
    for filename in os.listdir(folder+"/"+sub_folder):  #   Iterating through the files in the sub-folder.
        
        #   Extracting the features from the audio files.
        
        if filename.endswith(".mp3"):   #   Checking whether the audio file is in MP3 coding format or not.
            file=folder+"/"+sub_folder+"/"+filename
            label=filename
            data=extract_features(file)
            feature_dataframe=feature_dataframe.append({"Feature":data, "Label":label}, ignore_index=True)  #   Appending the features and corresponding labels to the pandas.DataFrame.
        else:
            continue

 25%|██▌       | 20/79 [06:18<21:59, 22.36s/it][src/libmpg123/layer3.c:INT123_do_layer3():1773] error: part2_3_length (3360) too large for available bit count (3240)
[src/libmpg123/layer3.c:INT123_do_layer3():1773] error: part2_3_length (3328) too large for available bit count (3240)
100%|██████████| 79/79 [31:43<00:00, 24.10s/it]


In [4]:
feature_dataframe

,Feature,Label
0,"[-154.95695, 112.73386, 21.022905, 40.27971, 6...",150018.mp3
1,"[-191.90166, 147.03711, 25.290674, 20.171871, ...",150268.mp3
2,"[-115.253654, 118.442, -29.659176, 30.92171, -...",150080.mp3
3,"[-155.35367, 121.62723, -35.02517, 31.631826, ...",150078.mp3
4,"[-103.724915, 111.12672, -6.402428, 24.632692,...",150079.mp3
...,...,...
3410,"[-180.77026, 109.15954, -28.646, 42.15213, 8.0...",149625.mp3
3411,"[-158.45363, 79.61905, 20.46484, 42.4414, 3.29...",149369.mp3
3412,"[-179.08354, 129.38995, 18.004972, 14.838847, ...",149185.mp3
3413,"[-202.11386, 102.882904, -30.251274, 26.861208...",149422.mp3


In [5]:
len(feature_dataframe)  #   Checking the number of audio files in the pandas.DataFrame.

3415

In [6]:
feature_dataframe.to_pickle("features.pkl") #   Saving the pandas.DataFrame as a pickle (.pkl) file.
feature_array=np.array(feature_dataframe.Feature.tolist())  #   Converting the features to a NumPy.NDArray.

In [7]:
feature_array.shape[1]  #   Checking the number of features in each NumPy.NDArray.

40

- **Creating the Locality-Sensitive Hashing (LSH) hash table using the appropriate number of tables and hash size:**

For the implementation of the Locality-Sensitive Hashing (LSH) hash table, the approximate nearest neighbour search algorithm is used, which return points, whose distance from the query is at most `c` times the distance from the query to its nearest points. In the approximate nearest neighbour search algorithm, the search starts only on a small subset of candidates points – MinHash, in the case of Locality-Sensitive Hashing (LSH).

In [8]:
#   Function to create and return the Locality-Sensitive Hashing (LSH) hash table using the AnnoyIndex class.

def create_annoy_index(mfcc, n_trees):
    index=annoy.AnnoyIndex(mfcc.shape[1], "angular")    #   Creating an AnnoyIndex object with the angular distance metric.
    for i, j in enumerate(mfcc):    #   Iterating through the Mel-Frequency Cepstral Coefficients (MFCC) features.
        index.add_item(i, j)    #   Adding the Mel-Frequency Cepstral Coefficients (MFCC) features to the AnnoyIndex object.
    index.build(n_trees)    #   Building the AnnoyIndex object with the specified number of trees.
    return index

In [9]:
annoy_index=create_annoy_index(feature_array, 100)
annoy_index.save("music.ann")   #   Saving the AnnoyIndex object into a separate file.

True

- **Creating the feature vector (Mel-Frequency Cepstral Coefficients (MFCC)) and running a query on the Locality-Sensitive Hashing (LSH) hash tables for a recently read audio:**

In [10]:
#   Function to return the approximate nearest neighbours of the recently read audio file.

def get_nearest_neighbours(new_audio, annoy_index, n_neighbours):
    new_audio_mfcc=extract_features(new_audio)  #   Extracting the Mel-Frequency Cepstral Coefficients (MFCC) features from the recently read audio file.
    nearest_neighbours=annoy_index.get_nns_by_vector(new_audio_mfcc, n_neighbours)  #   Retrieving the approximate nearest neighbours of the recently read audio file.
    return nearest_neighbours

- **Comparing the feature vector of the recently read audio with the matches returned by the Locality-Sensitive Hashing (LSH) hash tables:**

In [11]:
#   Function to return the cosine distances between the recently read audio file and its approximate nearest neighbours.

def compare_vectors(new_mfcc, mfcc, nearest_neighbours):
    distances=[]
    for neighbour in nearest_neighbours:    #   Iterating through the approximate nearest neighbours of the recently read audio file.
        distances.append(cosine(new_mfcc, mfcc[neighbour])) #   Calculating the cosine distances between the recently read audio file and its approximate nearest neighbours.
    return distances

- **Returning the result that has lowest/highest metric value (depending on the chosen metric) as the match:**

In [12]:
#   Function to return the best match (highest metric value) of the recently read audio file.

def get_best_match(new_audio, mfcc, annoy_index, n_neighbours):
    nearest_neighbours=get_nearest_neighbours(new_audio, annoy_index, n_neighbours) #   Retrieving the approximate nearest neighbours of the recently read audio file.
    distances=compare_vectors(extract_features(new_audio), mfcc, nearest_neighbours)    #   Calculating the cosine distances between the recently read audio file and its approximate nearest neighbours.
    best_match=nearest_neighbours[np.argmin(distances)] #   Retrieving the best match (highest metric value) of the recently read audio file.
    return best_match

In [13]:
new_audio=r"somebody_that_i_used.mp3"   #   Reading a new audio file.
audio, sample_rate=librosa.load(new_audio, res_type="kaiser_fast")  #   Loading the recently read audio file.
Audio(audio, rate=sample_rate)  #   Playing the recently read audio file.

In [14]:
best_match=get_best_match(new_audio, feature_array, annoy_index, 100)   #   Retrieving the best match (highest metric value) of the recently read audio file.
print(best_match)

2934


In [15]:
print(feature_dataframe.Label[best_match])

056499.mp3


In [16]:
#   Function to return the worst match (lowest metric value) of the recently read audio file.

def get_worst_match(new_audio, mfcc, annoy_index, n_neighbours):
    nearest_neighbours=get_nearest_neighbours(new_audio, annoy_index, n_neighbours) #   Retrieving the approximate nearest neighbours of the recently read audio file.
    distances=compare_vectors(extract_features(new_audio), mfcc, nearest_neighbours)    #   Calculating the cosine distances between the recently read audio file and its approximate nearest neighbours.
    worst_match=nearest_neighbours[np.argmax(distances)]    #   Retrieving the worst match (lowest metric value) of the recently read audio file.
    return worst_match

In [17]:
worst_match=get_worst_match(new_audio, feature_array, annoy_index, 100) #   Retrieving the worst match (lowest metric value) of the recently read audio file.
print(worst_match)

948


In [18]:
print(feature_dataframe.Label[worst_match])

007872.mp3


In [19]:
nearest_neighbours=get_nearest_neighbours(new_audio, annoy_index, 100)  #   Retrieving the approximate nearest neighbours of the recently read audio file.
distances=compare_vectors(extract_features(new_audio), feature_array, nearest_neighbours)   #   Calculating the cosine distances between the recently read audio file and its approximate nearest neighbours.

#   Printing the Local-Sensitive Hashing (LSH) hash table based on the cosine distances and the corresponding audio files.

for i in range(len(nearest_neighbours)):
    print(distances[i], "->", feature_dataframe.Label[nearest_neighbours[i]])

0.009337186813354492 -> 056499.mp3
0.009435415267944336 -> 055102.mp3
0.009974777698516846 -> 001196.mp3
0.010871827602386475 -> 026654.mp3
0.011160194873809814 -> 020818.mp3
0.01171112060546875 -> 037369.mp3
0.011970102787017822 -> 064008.mp3
0.012371599674224854 -> 043030.mp3
0.012517809867858887 -> 036322.mp3
0.01344907283782959 -> 021997.mp3
0.01346057653427124 -> 032686.mp3
0.013608038425445557 -> 014541.mp3
0.013699173927307129 -> 027798.mp3
0.013935863971710205 -> 148611.mp3
0.014003396034240723 -> 001925.mp3
0.014280498027801514 -> 003904.mp3
0.01431131362915039 -> 027804.mp3
0.014422357082366943 -> 063208.mp3
0.014492928981781006 -> 011839.mp3
0.014540433883666992 -> 067233.mp3
0.014686942100524902 -> 006463.mp3
0.01473933458328247 -> 070207.mp3
0.014787137508392334 -> 001193.mp3
0.014834761619567871 -> 029738.mp3
0.01490020751953125 -> 003906.mp3
0.014923810958862305 -> 148612.mp3
0.015131771564483643 -> 057176.mp3
0.015230536460876465 -> 149775.mp3
0.015284121036529541 -> 07

##### Question #2:

- **Audio Segmentation:**

Given an audio signal, the objective is to use Locality-Sensitive Hashing (LSH) to identify similar audio segments in the signal. This information can be used to divide the audio signal into meaningful segments, such as words or phrases, for further processing.

- Given a query audio segment, retrieve the most similar audio segments from the Locality-Sensitive Hashing (LSH) hash table index.
- Group the similar audio segments into clusters, based on their similarity – the K-means clustering algorithm is used for this.
- Based on the clusters obtained from the segment clustering, divide the audio signal into meaningful segments. For instance, each cluster can be considered as a separate segment.

In [20]:
#   Function to perform audio segmentation using the K-means clustering algorithm on the recently read audio file.

def audio_segmentation(new_audio, annoy_index, n_neighbours, n_clusters):
    neighbours=get_nearest_neighbours(new_audio, annoy_index, n_neighbours) #   Retrieving the approximate nearest neighbours of the recently read audio file.
    kmeans=KMeans(n_clusters=n_clusters, random_state=0, algorithm="elkan").fit(feature_array[neighbours])  #   Performing K-means clustering on the approximate nearest neighbours of the recently read audio file.
    cluster_labels=kmeans.labels_   #   Retrieving the cluster labels of the approximate nearest neighbours of the recently read audio file.
    cluster_dataframe=pd.DataFrame({"Cluster": cluster_labels, "Song": feature_dataframe.Label[neighbours]})    #   Creating a pandas.DataFrame from the cluster labels and the corresponding song names.
    cluster_dataframe=cluster_dataframe.sort_values(by=["Cluster"]) #   Sorting the pandas.DataFrame by the cluster labels.
    cluster_dataframe=cluster_dataframe.reset_index(drop=True)  #   Resetting the index of the pandas.DataFrame.
    cluster_dataframe.to_csv("pied_piper_download.csv", index=False)    #   Saving the pandas.DataFrame into a separate comma-separated values (CSV) file.
    return cluster_dataframe

In [21]:
clusters=audio_segmentation(new_audio, annoy_index, 100, 10)
clusters

,Cluster,Song
0,0,064515.mp3
1,0,052448.mp3
2,0,047897.mp3
3,0,056034.mp3
4,0,054151.mp3
...,...,...
95,9,056645.mp3
96,9,060510.mp3
97,9,001684.mp3
98,9,007373.mp3
